In [ ]:
import os
import time
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

from os import listdir
from os.path import isfile, join

from lxml import etree



# <<<---configuration--->>>

copy model to 'cropping_model' folder
copy images to 'img_for_cropping' folder

# <<<---configuration--->>>

In [ ]:
model_path = os.path.join('Tensorflow', 'crop_workspace','cropping_model')
MODEL_NAME = [name for name in os.listdir(model_path) if os.path.isdir(model_path)]

CUSTOM_MODEL_NAME = MODEL_NAME[0] 
LABEL_MAP_NAME = 'label_map.pbtxt'

print('Model: ',CUSTOM_MODEL_NAME)

In [ ]:
paths = {
    'MODEL_PATH': os.path.join('Tensorflow', 'crop_workspace','cropping_model'),
    'IMG_BEFORE_CROP_PATH': os.path.join('Tensorflow', 'crop_workspace','img_for_cropping'),
    'IMG_AFTER_CROP': os.path.join('Tensorflow', 'crop_workspace','cropped_img'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'crop_workspace','cropping_model',CUSTOM_MODEL_NAME),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'crop_workspace','cropping_model',CUSTOM_MODEL_NAME,'dev','annotations'),
 }

creating folders from path:

In [ ]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

In [ ]:
files = {
    'PIPELINE_CONFIG':os.path.join(paths['MODEL_PATH'], CUSTOM_MODEL_NAME, 'pipeline.config'),
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

Getting latest checkpoint from models folder:

In [ ]:
file_names_in_model_dir = os.path.join(paths['MODEL_PATH'],CUSTOM_MODEL_NAME)
ckpt_files = []
all_files_from_model_dir = [f for f in listdir(file_names_in_model_dir) if isfile(join(file_names_in_model_dir, f))] #lista nazw plikow
for i in all_files_from_model_dir :
    if i[-1]=='x': (ckpt_files.append(i))
print(ckpt_files)
numbers = []
temp_num = ""
# extract number of ckpt files:
for one_name in ckpt_files:
    for c in one_name:
        if c.isdigit(): # finding numbers in string
            temp_num = temp_num + c
    numbers.append(int(temp_num))
    temp_num=""
    max_num = max(numbers)#largest number from list
latest_checkpoint_name = "ckpt-"+str(max_num)  #name of latest checkpoint
print(CUSTOM_MODEL_NAME,'---------->',latest_checkpoint_name)

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'],latest_checkpoint_name)).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [ ]:
TEST_PATH = os.path.join(paths['IMG_BEFORE_CROP_PATH'])
RIGHT_DETECTION_FILE_NAME =[]
BAD_DETECTION_FILE_NAME =[]
detection_status=""

files = [f for f in listdir(TEST_PATH) if isfile(join(TEST_PATH, f))] #lista nazw plikow
files.sort()
jpg_images = [] #list for image names
for i in files:
    if i[-1]=='g':
        jpg_images.append(i)

koniec = True
        
for image_name in jpg_images:
    IMAGE_PATH = os.path.join(TEST_PATH, image_name)
    
    img = cv2.imread(IMAGE_PATH)
    img2 = img
    image_np = np.array(img)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=1,
                min_score_thresh=.1,
                agnostic_mode=False)
    
    #-----checking if detection is legit
#     image_name.rstrip('.jpg')
    do_nazwy = category_index[detections['detection_classes'][0]+label_id_offset] #wydobycie danej sekcji z nazwa
    finded_object_name = do_nazwy['name']



    if koniec:
        new_folder_path = paths['IMG_AFTER_CROP']
        img3 = img2.copy()
        height = img2.shape[0]
        width = img2.shape[1]
        img_alone_name = image_name.rstrip('.jpg')
        box = detections['detection_boxes'][0]
        
        
        x1=int(box[1]*width)
        y1=int(box[0]*height)
        x2=int(box[3]*width)
        y2=int(box[2]*height)
        
        xx1=x1-10
        yy1=int(((height-y1)/2)+y1)
        xx2=xx1+90
        yy2=yy1+220
        crp_image = img2[yy1:yy2, xx1:xx2]
        
        
        
# #     ---------zapis
        # if statement just in case if cropping go wrong and array is empty
        if crp_image.size > 0: 
            SAVE_PATH = os.path.join(new_folder_path, img_alone_name+'_cropped.jpg')
            cv2.imwrite(SAVE_PATH,  crp_image) 
# #     ---------zapis
            plt.imshow(cv2.cvtColor(crp_image, cv2.COLOR_BGR2RGB))
            plt.show()
        
        
#         print('x_min=', x1,' y_min=', y1,' x_max=', x2,' y_max=',y2)
#         plt.imshow(cv2.cvtColor(img3, cv2.COLOR_BGR2RGB))
#         plt.show()
#         print('---------------------------------------------------------------')


    
